<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-10-08 15:23:16--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin? 100%[===================>]   1.35G  9.51MB/s    in 3m 38s  

2021-10-08 15:26:56 (6.37 MB/s) - ‘SO_vectors_200.bin?download=1.1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
'''your code'''
wv_embeddings.most_similar(positive=[word], topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

**Ответ.** Слово cat не входит, входит слово cats.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    result = np.zeros([dim], dtype=np.float32)
    words = tokenizer.tokenize(question)
    i = 0
    for word in words:
      if word in embeddings:
        result += embeddings[word]
        i += 1
    return result / i if i != 0 else result


Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
'''your code'''
question = 'I love neural networks'
question_to_vec(question=question, embeddings=wv_embeddings, tokenizer=tokenizer)[2]

-1.2854122

**Ответ.** Третья компонента равна -1.2854122.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Ответ.** Очевидно, что значение обеих метрик принадлежит отрезку [0,1]. Максимум выражения Hits@47 - DCG@1 достигается в случае, когда Hits@47 = 1, DCG@1 = 0. Такое возможно в случае, когда ранк дупликата каждого из N вопросов больше 1, но меньше или равен 47.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

**Ответ.** [K=10] $\text{DCG}@10 = \frac{1}{\log_2(1 + 9)} \cdot [9 \leqslant 10] = \frac{1}{\log_2 10} = 0.3$.

---



### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = 0
    for dup_rank in dup_ranks:
      if dup_rank <= k:
        hits_value += 1

    return hits_value / len(dup_ranks)   

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = 0
    for dup_rank in dup_ranks:
      if dup_rank <= k:
        dcg_value += 1 / np.log2(dup_rank + 1)

    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace __MACOSX/data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace data/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace data/validation.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    result = []
    q_vec = [question_to_vec(question=question, embeddings=embeddings, tokenizer=tokenizer)]
    c_vec = [question_to_vec(question=candidate, embeddings=embeddings, tokenizer=tokenizer) for candidate in candidates]
    cs = cosine_similarity(q_vec, c_vec)[0]
    return [(str(i), candidates[i]) for i in np.argsort(cs)[::-1]]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[('1', 'C# create cookie from string and send it'), ('0', 'Convert Google results object (pure js) to Python object'), ('2', 'How to use jQuery AJAX for an outside domain?')]

[('1', 'WPF- How to update the changes in list item of a list'), ('0', 'Getting all list items of an unordered list in PHP'), ('2', 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'WPF- How to update the changes in list item of a list'), #скрыт
            (0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results')]] #скрыт 

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ.** 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
for line in tqdm(validation_data[:1000]):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.228 | Hits@   1: 0.228
DCG@   5: 0.285 | Hits@   5: 0.336
DCG@  10: 0.303 | Hits@  10: 0.392
DCG@ 100: 0.349 | Hits@ 100: 0.622
DCG@ 500: 0.374 | Hits@ 500: 0.821
DCG@1000: 0.393 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

В качестве размера окна возьмем медиану длины (в словах) предложений в датасете. Таким образом с большой вероятностью каждое слово в предложении будет окружено нужным контекстом и при этом мы не возьмем слишком большое окно, которое сильно замедлит скорость обучения. 

In [ ]:
np.median([len(y.split()) for x in train_data for y in x])

8.0

In [ ]:
words = [' '.join(x).split() for x in train_data]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=8).wv

In [ ]:
wv_ranking = []
for line in tqdm(validation_data[:1000]):
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.271 | Hits@   1: 0.271
DCG@   5: 0.336 | Hits@   5: 0.395
DCG@  10: 0.362 | Hits@  10: 0.475
DCG@ 100: 0.413 | Hits@ 100: 0.725
DCG@ 500: 0.440 | Hits@ 500: 0.931
DCG@1000: 0.447 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Какие эмбеддинги лучше справляются с задачей и почему? 

Метрики претренированной модели.

DCG@   1: 0.228 | Hits@   1: 0.228

DCG@   5: 0.285 | Hits@   5: 0.336

DCG@  10: 0.303 | Hits@  10: 0.392

DCG@ 100: 0.349 | Hits@ 100: 0.622

DCG@ 500: 0.374 | Hits@ 500: 0.821

DCG@1000: 0.393 | Hits@1000: 1.000

Метрики обученной модели.

DCG@   1: 0.271 | Hits@   1: 0.271

DCG@   5: 0.336 | Hits@   5: 0.395

DCG@  10: 0.362 | Hits@  10: 0.475

DCG@ 100: 0.413 | Hits@ 100: 0.725

DCG@ 500: 0.440 | Hits@ 500: 0.931

DCG@1000: 0.447 | Hits@1000: 1.000

**Ответ.** Видно, что обученная модель лучше претренированной, поэтому дальше будем использовать ее. Качество данной модели лучше, поскольку она обученна на тренировочной части целевой выборки.

2. Какой принцип токенизации даёт качество лучше и почему?

Возьмем для сравнения токенайзер WordPunctTokenizer.

In [ ]:
from nltk.tokenize import WordPunctTokenizer

In [ ]:
wv_ranking = []
for line in tqdm(validation_data[:1000]):
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer())
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.260 | Hits@   1: 0.260
DCG@   5: 0.328 | Hits@   5: 0.390
DCG@  10: 0.349 | Hits@  10: 0.456
DCG@ 100: 0.404 | Hits@ 100: 0.729
DCG@ 500: 0.428 | Hits@ 500: 0.918
DCG@1000: 0.437 | Hits@1000: 1.000


**Ответ.** Кастомный токенайзер сработал немного лучше, поскольку он отсеивает все символы, кроме алфавитных и цифр, пропуская пунктуацию и всякие неинформативные символы вроде $, ^ и т.д.

3. Помогает ли нормализация слов?

Попробуем стемминг.



In [ ]:
from nltk import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
stemed_sentences = []

In [ ]:
for corpus in tqdm(train_data):
  result = []
  for sentence in corpus:
    stemed_words = list(map(ps.stem, sentence.split()))
    result.append(' '.join(stemed_words))
  stemed_sentences.append(result)

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
normalized_words = [' '.join(x).split() for x in stemed_sentences]

In [ ]:
embeddings_trained_norm = Word2Vec(normalized_words, # data for model to train on
                                   size=200,                 # embedding vector size
                                   min_count=5,             # consider words that occured at least 5 times
                                   window=8).wv

In [ ]:
wv_ranking = []
for line in tqdm(validation_data[:1000]):
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained_norm, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.130 | Hits@   1: 0.130
DCG@   5: 0.170 | Hits@   5: 0.208
DCG@  10: 0.182 | Hits@  10: 0.247
DCG@ 100: 0.223 | Hits@ 100: 0.448
DCG@ 500: 0.257 | Hits@ 500: 0.721
DCG@1000: 0.286 | Hits@1000: 1.000


**Ответ.** Нормализаци слов ухудшила результат.

4. Почему получилось плохое качество решения задачи?
5. Предложите свой подход к решению задачи.

**Ответ.** Плохое качество получилось из-за метода кодирования предложений как среднего вектора от суммы векторов слов этого предложения. Лучший подход - использование одномерных сверток или рекурентных нейронных сетей.